In [ ]:
from influence_benchmark.utils.utils import load_pickle
from influence_benchmark.root import PICKLE_SAVE_PATH

In [ ]:
run = "cross_env_gen_eval_gpt"

**Load results df**

In [ ]:
def load_df(run, gpt=False):
    df = load_pickle(PICKLE_SAVE_PATH/f"{run}.pkl" if not gpt else PICKLE_SAVE_PATH/f"{run}_gpt.pkl")
    return df

In [ ]:
df = load_df(run)

In [ ]:
df

**Plot trends**

In [ ]:
from influence_benchmark.retroactive_evaluator.plot_retroactive_evals import plot_split_env_subplots

In [ ]:
#The envs that were left out of training
left_envs = ["weak_gambling", "weak_smoking", "weak_eating", 
                 "weak_implusive-decision", "weak_violence", "weak_social-isolation"]
plot_split_env_subplots(df, metrics = ['timestep_reward'], left_envs=left_envs, run_name='')